In [1]:
import urllib
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup
from datetime import datetime

In [136]:
#a random thread starter
#url = 'https://www.notaryrotary.com/forums/forums.asp?id=&forumid=1AAA00000003&messageid=2081512&code=&smsg=&requestid=&action=view&format=threaded'
url = 'https://www.notaryrotary.com/forums/forums.asp?id=&forumid=1AAA00000003&messageid=2081526&code=&smsg=&requestid=&action=view&format=threaded'

In [128]:
#process the URL and turn it into soup
def make_soup(my_url):
    req = urllib.request.Request(my_url)
    response = urllib.request.urlopen(req)
    thePage = response.read()
    return BeautifulSoup(thePage, 'lxml')

In [134]:
'''grab post data'''

def grab_post(my_url, my_soup):
    #get forum_id
    parsed_url = urlparse(my_url)
    query = parse_qs(parsed_url.query)
    my_forum_id = query['forumid'][0]
    my_message_id = int(query['messageid'][0])

    #get title
    my_title = my_soup.find('title').get_text()

    #get content
    my_content = my_soup.find(class_ = 'ForumText').get_text()

    #get author.  There are multiple <b>, but the first is the author
    my_author = my_soup.find('b').get_text()

    #get timestamp
    my_time = my_soup.find('em').get_text()
    my_time = my_time.strip(' ')
    my_time_real = datetime.strptime(my_time, '%m/%d/%y %I:%M%p')
    my_time = datetime.strftime(my_time_real, '%Y-%m-%d %H:%M:00')

    print(my_forum_id + ' - ' + str(my_message_id))
    print(my_title + ' - ' + my_author + ' - ' + my_time)
    print(my_content)

In [130]:
'''direct children only, no grandchildre'''
def list_children(my_soup):

    #TDAlt9 is unique for the messages table, specifically a cell in it
    t = my_soup.find(class_ = 'TDAlt9')
    thread_table = t.parent.parent
    for r in thread_table.find_all('tr'):
        i = r.find('img')
        try:   #first row has no image
            if (i['src'] == '/images/sp1.gif'):   #children are sp1, grandchildren sp2, etc
                ra = r.find('a', class_='A2')     #profiles have links, but not A2 class
                rah = ra['href']
                parsed_rah = urlparse(rah)
                rah_query = parse_qs(parsed_rah.query)
                print(rah_query['forumid'], rah_query['messageid'])
        except:
            continue

In [137]:
soup = make_soup(url)
print('soup made')
print('')
print('vvvv Post data vvvvv')
grab_post(url, soup)
print('')
print('vvvv Post children vvvvv')
list_children(soup)

soup made

vvvv Post data vvvvv
1AAA00000003 - 2081526
Sorry for the typos... n/m - MikeC/TX - 2017-05-25 16:27:00


vvvv Post children vvvvv
['1AAA00000003'] ['2081515']
['1AAA00000003'] ['2081518']
['1AAA00000003'] ['2081524']
['1AAA00000003'] ['2081540']
['1AAA00000003'] ['2081544']
['1AAA00000003'] ['2081603']


# Scratch

In [12]:
class Post:
    def __init__(self, forum_id, message_id):
        self.forum_id = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick):
        self.tricks.append(trick)

In [16]:
soup

<html><head><meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<title>PDF files scanned are too large to send in emails. How can..</title><meta content="no-cache" http-equiv="Pragma"/><meta content="-1" http-equiv="Expires"/><meta content="Notary Rotary, Inc." name="author"/><meta content="© 2002-2017, Notary Rotary, Inc." name="copyright"/><meta content="Business and Economy" name="classification"/><meta content="General" name="rating"/><meta content="15 days" name="Revisit-After"/><meta content="PDF files scanned are too large to send in emails. How can.." name="description"/><meta content="PDF files scanned are too large to send in emails. How can.." name="keywords"/><meta content="English" http-equiv="Content-Language"/><meta content="document" name="resource-type"/><meta content="2UIYLRAXt4l-90dMVvdHqr-xHQYCwROur90xV5afAaE" name="google-site-verification"/><link href="/include/body.css" rel="stylesheet" type="text/css"/><script type="text/java

['1AAA00000003'] ['2081515']
['1AAA00000003'] ['2081518']
['1AAA00000003'] ['2081524']
['1AAA00000003'] ['2081540']
['1AAA00000003'] ['2081544']


In [110]:
t = soup.find(class_ = 'TDAlt9')
thread_table = t.parent.parent
for r in thread_table.find_all('a', class_='A2'):
    rh = r['href']
    parsed_rh = urlparse(rh)
    rh_query = parse_qs(parsed_rh.query)
    print(rh_query['forumid'], rh_query['messageid'])

<tr><td align="center" nowrap=""><img height="13" src="/images/triangle.gif" width="12"/></td><td nowrap=""><img src="/images/b0.gif"/> <a class="A2" href="/forums/forums.asp?id=&amp;forumid=1AAA00000003&amp;messageid=2081512&amp;code=&amp;smsg=&amp;requestid=&amp;action=view&amp;format=threaded">PDF files scanned are too large to send in emails. How can..</a> - <font color="#333333" size="2"><b>Susan/CA</b></font><font color="#333333"><em> on 5/25/17 3:09pm  </em></font></td></tr>
<tr><td align="center" nowrap=""></td><td nowrap=""><img src="/images/sp1.gif"/><img src="/images/b0.gif"/> <a class="A2" href="/forums/forums.asp?id=&amp;forumid=1AAA00000003&amp;messageid=2081515&amp;code=&amp;smsg=&amp;requestid=&amp;action=view&amp;format=threaded">Re: PDF files scanned are too large to send in emails. How can..</a> - <font color="#333333" size="2"><b><img border="0" height="10" src="/images/lvl1.gif" width="10"/> <a class="TDAlt7" href="/viewprofile.asp?id=&amp;RID=0116776F851A89E1D874E